In [1]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [2]:
train_set = pd.read_csv('train/train_encoded_sin_location.csv')
train_set.head(10)

,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,keyword_8,id,len_text,text_sum,text_min,text_max,text_mean,text_median,target
0,0,0,0,0,0,0,0,1,1,43,2.745943,0.224894,0.474996,0.343243,0.323476,1
1,0,0,0,0,0,0,0,1,4,36,2.545323,0.211650,0.508967,0.363618,0.340166,1
2,0,0,0,0,0,0,0,1,5,72,3.925010,0.153758,0.585124,0.327084,0.256037,1
3,0,0,0,0,0,0,0,1,6,50,2.515355,0.150085,0.574226,0.359336,0.346962,1
4,0,0,0,0,0,0,0,1,7,52,2.937988,0.247769,0.444024,0.326443,0.287533,1
5,0,0,0,0,0,0,0,1,8,84,3.472170,0.094812,0.392155,0.267090,0.268381,1
6,0,0,0,0,0,0,0,1,10,74,3.652263,0.211903,0.421501,0.304355,0.292368,1
7,0,0,0,0,0,0,0,1,13,21,2.169400,0.288043,0.580492,0.433880,0.410864,1
8,0,0,0,0,0,0,0,1,14,38,2.425554,0.344773,0.491582,0.404259,0.389401,1
9,0,0,0,0,0,0,0,1,15,24,1.966376,0.391599,0.639305,0.491594,0.467736,1


### XGBoost

In [3]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Random Search

In [15]:
learning_rate = [0.1, 0.3, 0.01, 0.001]
max_depth = [20, 50, 70, 100]
#subsamples = [0.4, 0.5, 0.6]
colsample_bytree = [0.4, 0.5, 0.6]
n_estimators = [20, 50, 70, 100]
objective = ['reg:logistic']

random_search = {'learning_rate': learning_rate, 'max_depth': max_depth,\
                'colsample_bytree': colsample_bytree, 'n_estimators': n_estimators, 'objective': objective}

In [16]:
xgb_random = RandomizedSearchCV(estimator = xgb.XGBClassifier(),\
                               param_distributions=random_search, n_iter=50, cv=7, random_state=111)

In [17]:
xgb_random.fit(X_train, y_train)

RandomizedSearchCV(cv=7,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                                    

In [18]:
xgb_random.best_params_

{'objective': 'reg:logistic',
 'n_estimators': 100,
 'max_depth': 100,
 'learning_rate': 0.01,
 'colsample_bytree': 0.5}

In [20]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.01, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
acc = np.sqrt(accuracy_score(y_test, preds))
print("Accuracy Score: %f" % (acc))

Accuracy Score: 0.841409


##### Best params de random forest

In [7]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3, colsample_bytree = 0.6,\
                           learning_rate = 0.1, max_depth = 70, alpha = 10, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
acc = np.sqrt(accuracy_score(y_test, preds))
print("Accuracy Score: %f" % (acc))

Accuracy Score: 0.809586


##### Parámetros que mejor funcionaron con pruebas a mano

In [6]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3,
                colsample_bytree = 0.6, learning_rate = 0.1,
                max_depth = 70, alpha = 10, n_estimators = 57)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
acc = np.sqrt(accuracy_score(y_test, preds))
print("Accuracy Score: %f" % (acc))

Accuracy Score: 0.801706


### Test

In [21]:
test_set = pd.read_csv('test/test_encoded_sin_location.csv')

In [22]:
preds = xg_reg.predict(test_set)
preds

array([0, 1, 1, ..., 0, 1, 1])

In [23]:
test_set['target'] = preds
test_set[['id', 'target']]

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,0
3261,10874,1


### Resultado

In [24]:
test_set[['id', 'target']].to_csv('result_xgb_sin_location.csv', header=True, index=False)